# Intro to Gen AI Orchestration - Basics

Use LLM API to build simple applications.

## 1. Use OpenAI API

In the first example, we will use OpenAI API directly to build a simple chat interface.

🛠 Install python libraries

In [7]:
%%capture
!pip install openai  # OpenAI python SDK
!pip install gtts    # Google text to speech
!pip install mmh3    # Murmur3 hash function

Read the API key securely as a secret using colab SDK.

In [27]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Demo 1.a: OpenAI Chat Completions API

💬 **Chat Completions** example from OpenAI quickstart tutorial at https://platform.openai.com/docs/quickstart

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

🔍 Inspect the response text.

In [4]:
n = 4
content = completion.choices[0].message.content
lines = content.split('\n')
top_n_lines = '\n'.join(lines[:n])

print(top_n_lines)

In the realm of code's endless dance,
There lies a fascinating trance,
Where functions call themselves with glee,
In a recursive reverie.


In [ ]:
from pathlib import Path
from openai import OpenAI

message_content = top_n_lines

message_hash = mmh3.hash_bytes(message_content).hex()
audio_filename = f"{message_hash}.mp3"

response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=message_content
)

response.stream_to_file(audio_filename)

📢 Read out the response using TTS

In [22]:
import mmh3
from gtts import gTTS
from io import BytesIO
from IPython.display import Audio


# message_content = completion.choices[0].message.content
message_content = top_n_lines

message_hash = mmh3.hash_bytes(message_content).hex()
audio_filename = f'{message_hash}.wav'

tts = gTTS(message_content)

# check if audio_filename already exists
if not os.path.exists(audio_filename):
  tts.save(audio_filename)

Audio(audio_filename)

💬 Continue the chat by asking a folllow up question.

Since the chat completions API is stateless, we need to pass the full conversation in order to preserve the context.

In [ ]:
follow_up = "Convert this poem to a haiku. Preserve the meaning by condensing as needed."

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
    {"role": "assistant", "content": content},
    {"role": "user", "content": follow_up}
  ]
)

haiku = completion.choices[0].message.content
print(haiku)